<a href="https://colab.research.google.com/github/jinsusong/study-NLP-BERT/blob/main/QA_SQuAD1_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pwd

In [ ]:
%cd /content/drive/MyDrive/data/squad1/
%pwd

In [ ]:
!ls -al

# 데이터 로드

In [ ]:
!pip install datasets

In [ ]:
# Let's import the library. We typically only need at most four methods:
from datasets import list_datasets, list_metrics, load_dataset, load_metric

from pprint import pprint

In [ ]:
# Currently available datasets and metrics
#datasets = list_datasets()
#metrics = list_metrics()

# print(f" Currently {len(datasets)} datasets are available on the hub:")
# pprint(datasets, compact=True)
# print(f" Currently {len(metrics)} metrics are available on the hub:")
# pprint(metrics, compact=True)

In [ ]:
# You can access various attributes of the datasets before downloading them
#squad_dataset = list_datasets(with_details=True)[datasets.index('squad')]

#pprint(squad_dataset.__dict__)  # It's a simple python dataclass

In [ ]:
# Downloading and loading a dataset
dataset = load_dataset('squad', split='validation[:10%]')

In [ ]:
pprint(dataset.info.__dict__)

In [ ]:
print(dataset)
print(dataset['answers'])

In [ ]:
print(f"Dataset len(dataset): {len(dataset)}")
print("\nFirst item 'dataset[0]':")
pprint(dataset[0])

In [ ]:
print("\nSlice of the two items 'dataset[10:12]':")
pprint(dataset[10:12])

In [ ]:
# You can get a full column of the dataset by indexing with its name as a string:
print(dataset['question'][:10])

In [ ]:
dataset.map(lambda example: print(len(example['context']), end=','))

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizerFast, logging as transformers_logging

transformers_logging.set_verbosity_warning()

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [ ]:
dataset

In [ ]:
tokenizer(dataset['context'][0])

In [ ]:
# Now let's batch tokenize our dataset 'context'
encoded_dataset = dataset.map(lambda example: tokenizer(example['context']), batched=True)

print("encoded_dataset[0]")
pprint(encoded_dataset[0], compact=True)

In [ ]:
# Let show a more complex processing with the full preparation of the SQuAD dataset
# for training a model from Transformers
def convert_to_features(batch):
    # Tokenize contexts and questions (as pairs of inputs)
    input_pairs = list(zip())
    encodings = tokenizer(batch['context'], batch['question'], truncation=True)

    # Compute start and end tokens for labels
    start_positions, end_positions = [], []
    for i, answer in enumerate(batch['answers']):
        first_char = answer['answer_start'][0]
        last_char = first_char + len(answer['text'][0]) - 1
        start_positions.append(encodings.char_to_token(i, first_char))
        end_positions.append(encodings.char_to_token(i, last_char))

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return encodings

encoded_dataset = dataset.map(convert_to_features, batched=True)

In [ ]:
# Now our dataset comprise the labels for the start and end position
# as well as the offsets for converting back tokens
# in span of the original string for evaluation
print("column_names", encoded_dataset.column_names)
print("start_positions", encoded_dataset[:5]['start_positions'])